## Introducción a la Visión Computacional
## Tarea 2
-----------------------
<div align="right">
Fecha de Entrega: Viernes 29, Abril 2022.
</div>

1. Diseñar un flujo de procesamiento a nivel abstracto: adquisición de imágenes, pre-procesamiento, procesamiento de imágenes, cálculo de la medición, almacenamiento.

2. Con la base de datos seleccionada, implementar un sistema de reconocimiento y/o procesamiento en el cual se realice una medida o inferencia sobre imagen. Puede ser la detección de un objeto, medición de distancias entre píxeles de alguna característica, medición del tamaño de algún objeto (en cantidad de píxeles) clasificación de la imagen, entre otros.

3. Para procesar el conjunto completo de imágenes, medir los tiempos de ejecución del proceso completo. Realizar la medición para el 50% y el 100% de la imágenes varias veces, de manera de obtener algunas figuras estadísticas: tiempo máximo, tiempo mínimo, tiempo medio, desviación estándar del tiempo.

4. Hacer resumen de los resultados del procesamiento. Para ello debe decidir una métrica para indicar si el procesamiento es satisfactorio. Para esto último algunas veces es necesario hacer la evaluación en forma manual, por lo tanto, si fuera este el caso, seleccionar un sub-conjunto de las imágenes (por ejemplo, unas 50) y comparar el desempeño del algoritmo en estas imágenes.

5. Escribir un informe o entregar el notebook con comentarios y documentación, incluyendo el análisis (comentando) los resultados.

Nota: de percatarse que el set de imágenes propuesto en la Tarea 1 no es el adecuado, se puede realizar un cambio en el dataset. Justificar el cambio. Este tipo de decisiones puede ocurrir en un proyecto.

# Base de datos de patentes de vehiculos de Chile

Base de datos recopilada por mi, en imagenes de Google, las cuales fueran fotografias de patentes de vehiculos chilenos, con esto me gustaria realizar un posterior analisis, obteniendo las letras y numeros para consultar en [autoseguro.gob.cl](https://www.autoseguro.gob.cl) de carabineros y policia de investigaciones, con el objetivo de verificar si un vehiculo tiene encargo por robo.

En este caso para el pre-procesamiento de imagenes, se aplica el filtro sobel, para poder obtener la deteccion de bordes, con esto podemos tener la capacidad de detectar medianamente bien las letras y numeros de las patentes. Obteniendo estos datos, podria verificar si estos estan con encargo por robo y asi poder informar que el vehiculo fue visto por ultima vez en una locacion especifica

## Definicion funciones
Define funciones intermedias para ser referenciadas desde funcion de ejecucion de pipeline
- obtencion de imagenes desde classpath de proyecto ```get_image_list_from_path(path, extension)```
- pre-procesamiento de imagenes obtenidas en punto anterior con ```save_resized_images_resx_resy```
- procesamiento de imagenes con filtro sobel ```sobel_filter```

In [107]:
import os
from PIL import Image
from numpy import *
from scipy.ndimage import sobel
from matplotlib import pyplot as plt
import cv2

In [142]:
'''
Obtiene las imagenes definidas en el classpath del proyecto, con los formatos que se establecen en el segundo parametro
    - path: direccion donde estaran alojadas las imagenes a procesar
    - extension: lista de extensiones que seran tomadas en cuenta para obtencion de imagenes
'''
def get_image_list_from_path(path, extension):
    l = []
    for ex in extension:
        for f in os.listdir(path):
            if f.endswith(ex):
                l.append(os.path.join(path,f))

    return l

In [143]:
img_from_path = get_image_list_from_path("patentes", [".png", ".jpeg"])

In [144]:
'''
Guarda las imagenes que ya fueron guardadas en el classpath, con una resolucion diferente
    - x: resolucion al eje x
    - y: resolucion al eje y
    - img_list: lista de imagenes que seran procesadas a la resolucion pasada en los parametros x e y
'''
def save_resized_images_resx_resy(x, y, img_list):
    os.makedirs("out{}x{}/patentes".format(x, y), exist_ok=True)

    for img in img_list:
        Image.open(img).convert('RGB').resize((x, y))\
            .save("out{}x{}/{}".format(x, y, img))

    return get_image_list_from_path("out{}x{}/patentes".format(x, y), [".png", ".jpeg"])

In [145]:
resized_images_512_512 = save_resized_images_resx_resy(512, 512, img_from_path)

In [146]:
def sobel_filter(img_list):

    os.makedirs("out_sobel/patentes", exist_ok=True)

    for img in img_list:
        im = array(Image.open(img).convert('L'), 'f')
        imx = zeros(im.shape)
        sobel(im, 1, imx) #Filas (X)

        imy = zeros(im.shape)
        sobel(im, 0, imy) #Columnas (Y)

        magnitude = sqrt(imx**2+imy**2)
        plt.imsave("out_sobel/{}".format("/".join(img.split("/")[-2:])), arr = magnitude, cmap = plt.get_cmap('gray'))

    return get_image_list_from_path("out_sobel/patentes", [".png", ".jpeg"])

In [147]:
sobel_filter_images = sobel_filter(resized_images_512_512)

In [198]:
def detect_vehicle_license_plate(img_list):

    os.makedirs("out_detection/patentes", exist_ok=True)
    for img_path in img_list:

        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, threshold = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(
            threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        i = 0
        for contour in contours:
            if i == 0:
                i = 1
                continue
            approx = cv2.approxPolyDP(
                contour, 0.01 * cv2.arcLength(contour, True), True)
            cv2.drawContours(img, [contour], 0, (0, 0, 255), 5)
            M = cv2.moments(contour)
            if M['m00'] != 0.0:
                x = int(M['m10']/M['m00'])
                y = int(M['m01']/M['m00'])

        cv2.imwrite('out_detection/{}'.format("/".join(img_path.split("/")[-2:])), img)

In [199]:
detect_vehicle_license_plate(sobel_filter_images)

In [53]:
import cv2
import imutils
import numpy as np
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'tesseract'

img = cv2.imread('out_sobel/patentes/2.jpeg',cv2.IMREAD_COLOR)
img = cv2.resize(img, (600,400) )

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 13, 15, 15)

edged = cv2.Canny(gray, 30, 200)
contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
screenCnt = 0

for c in contours:

    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)

    if len(approx) == 4:
        screenCnt = approx
        break

if screenCnt is None:
    detected = 0
    print ("No contour detected")
else:
     detected = 1

if detected == 1:
    cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

mask = np.zeros(gray.shape,np.uint8)
#new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
#new_image = cv2.bitwise_and(img,img,mask=mask)

(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
Cropped = gray[topx:bottomx+1, topy:bottomy+1]

text = pytesseract.image_to_string(Cropped, config='--psm 11')
print("programming_fever's License Plate Recognition\n")
print("Detected license plate Number is:",text)
img = cv2.resize(img,(500,300))
Cropped = cv2.resize(Cropped,(400,200))
cv2.imshow('car',img)
cv2.imshow('Cropped',Cropped)

cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.5.5) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/drawing.cpp:2502: error: (-215:Assertion failed) npoints > 0 in function 'drawContours'
